In [53]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import datetime
import os
import csv
import io
import random
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score

In [54]:
ds = pd.read_csv('data.csv')
ds = ds.sample(frac=1).reset_index(drop=True)
# filter input/output factor(s)
ds_x = ds.iloc[:, [1,2,3,4,5]]
ds_y = ds.iloc[:,[-1]]
# 80% training
ds_train_x = ds_x[0:int(0.8*len(ds_x))]
ds_train_x = np.array(ds_train_x)
ds_train_y = ds_y[0:int(0.8*len(ds_y))]
# 20% testing
ds_test_x = ds_x[int(0.8*len(ds_x)):]
ds_test_x = np.array(ds_test_x)
ds_test_y = ds_y[int(0.8*len(ds_y)):]
# feature scaling (normalization)
sc = StandardScaler()
ds_train_x = sc.fit_transform(ds_train_x)
ds_test_x = sc.transform(ds_test_x)
ds_train_y = np.array(ds_train_y)
ds_test_y = np.array(ds_test_y)
temp = []
for i in ds_train_y:
    temp.append(i[0])
ds_train_y = np.array(temp)
temp = []
for i in ds_test_y:
    temp.append(i[0])
ds_test_y = np.array(temp)

In [55]:
epochs = 300
model = keras.Sequential()
model.add(keras.layers.Dense(450, activation='relu', input_dim=5))
#model.add(keras.layers.Dense(10))
#model.add(keras.layers.Dense(10))
model.add(keras.layers.Dense(1,activation='sigmoid')) # output layer; gives a 1D output
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = model.fit(ds_train_x, ds_train_y, epochs=epochs, verbose=0)
# apply to predictions
predictions = model.predict(ds_test_x)
predictions = predictions.reshape(len(predictions),)
diff = ds_test_y - predictions
corr_m = np.corrcoef(predictions, ds_test_y)
r2 = corr_m[0,1] ** 2
print('R2 = ' + str(r2*100) + "%")
mse = np.square(diff).mean()
print('MSE = ' + str(mse))
mae = np.mean(np.abs(diff))
print('MAE = ' + str(mae))

R2 = 99.81131848620197%
MSE = 0.0003806331032983539
MAE = 0.004779207864196585


In [56]:
#plt.plot(history.history['accuracy'])

In [57]:
# binary classification (from regression)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
ac = accuracy_score(ds_test_y,predictions)
cm = confusion_matrix(ds_test_y, predictions)
print('Accuracy: '+str(cm[0][0]+cm[1][1])+'/'+str(len(predictions))+' = '+str(ac*100)+'%')
print('No outbreak occurred, no outbreak predicted: '+str(cm[0][0])+'/'+str(len(predictions)))
print('No outbreak occurred, outbreak predicted: '+str(cm[0][1])+'/'+str(len(predictions)))
print('Outbreak occurred, no outbreak predicted: '+str(cm[1][0])+'/'+str(len(predictions)))
print('Outbreak occurred, outbreak predicted: '+str(cm[1][1])+'/'+str(len(predictions)))

Accuracy: 34/34 = 100.0%
No outbreak occurred, no outbreak predicted: 28/34
No outbreak occurred, outbreak predicted: 0/34
Outbreak occurred, no outbreak predicted: 0/34
Outbreak occurred, outbreak predicted: 6/34
